In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
import os
import xlrd
import time
import io
import matplotlib.pyplot as plt

In [2]:
dbprice=pd.read_csv('price.csv')
dbprice.set_index('ticker', inplace=True)
dbper=pd.read_csv('per.csv')
dbper.set_index('ticker', inplace=True)
dbpbr=pd.read_csv('pbr.csv')
dbpbr.set_index('ticker', inplace=True)
dbmarket=pd.read_csv('market.csv')
dbmarket.set_index('ticker', inplace=True)

FileNotFoundError: File b'market.csv' does not exist

In [ ]:
def yields(db):
    db['yield']=np.round((db[db.columns[2]]-db[db.columns[1]])/db[db.columns[1]]*100,2)
    return db

In [ ]:
def per_modeling(y, m, n):
    dbpermodel = dbper.iloc[:, [0,(y%7)*12+((y//7)-1)*84+m]]
    dbpermodel = dbpermodel.sort_values(by=dbpermodel.columns[1], ascending=True).head(100)
    dbpricemodel = dbprice.ix[dbpermodel.index, [0,(y%7)*12+((y//7)-1)*84+m,(y%7)*12+((y//7)-1)*84+m+12]]
    dbpricemodel[dbpricemodel.columns[1:3]] = dbpricemodel[dbpricemodel.columns[1:3]].astype(float)
    dbpricemodel = dbpricemodel[(dbpricemodel[dbpricemodel.columns[1]] > 0) & (dbpricemodel[dbpricemodel.columns[2]] > 0)].head(n)

    dbpricemodel=yields(dbpricemodel)
    return dbpricemodel

In [ ]:
per_modeling(7, 1, 10)

In [ ]:
dbpricemodel_m=per_modeling(7,1,10)['yield'].mean()
dbpricemodel_m

In [ ]:
def results(y, m, n):
    dbmarketmodel = dbmarket.iloc[:, [0,(y%7)*12+((y//7)-1)*84+m,(y%7)*12+((y//7)-1)*84+m+12]]
    dbmarketmodel = yields(dbmarketmodel)
    resultcolumns = [per_modeling(y,m,n)['yield'].mean(), dbmarketmodel.iloc[0, 3], dbmarketmodel.iloc[1,3]]
    resultindex = ['model','S&P500','NASDAQ']
    result = pd.DataFrame({'ticker':resultindex,'%s-%s'%(y, m):resultcolumns})
    result['%s-%s'%(y,m)]=resultcolumns
    results=result.set_index('ticker')
    return results

In [ ]:
def per_results(n):
    per_results = results(7, 1, n)
    for i in range(7,17):
        if i == 16:
            for j in range(1,6):
                per_results['%s-%s'%(i, j)]=results(i, j, n)['%s-%s'%(i, j)]
        else:
            for j in range(1,13):
                per_results['%s-%s'%(i, j)]=results(i, j, n)['%s-%s'%(i, j)]
    return per_results


In [ ]:
per_results(10)

In [ ]:
def win(a):
    win = 0
    for i in range(0, 113):
        if a.ix[0,i]>a.ix[1:3,i].mean():
            win += 1
    return win

def geomean(*numbers):
    a=1
    for i in numbers:
        a = a*i
    b= a**(1/len(numbers))
    return b

def change1(a):
    b = a/100 + 1
    return b

def change2(a):
    b = a*100 -100
    return b

In [ ]:
def result_mean(a):
    a1=change2(geomean(change1(a.ix[0, 0]),change1(a.ix[0, 12]),change1(a.ix[0, 24]),change1(a.ix[0, 36]),change1(a.ix[0, 48]),change1(a.ix[0, 60]),change1(a.ix[0, 72]),change1(a.ix[0, 84]),change1(a.ix[0, 96]),change1(a.ix[0, 108])))
    a2=change2(geomean(change1(a.ix[0, 1]),change1(a.ix[0, 13]),change1(a.ix[0, 25]),change1(a.ix[0, 37]),change1(a.ix[0, 49]),change1(a.ix[0, 61]),change1(a.ix[0, 73]),change1(a.ix[0, 85]),change1(a.ix[0, 97]),change1(a.ix[0, 109])))
    a3=change2(geomean(change1(a.ix[0, 2]),change1(a.ix[0, 14]),change1(a.ix[0, 26]),change1(a.ix[0, 38]),change1(a.ix[0, 50]),change1(a.ix[0, 62]),change1(a.ix[0, 74]),change1(a.ix[0, 86]),change1(a.ix[0, 98]),change1(a.ix[0, 110])))
    a4=change2(geomean(change1(a.ix[0, 3]),change1(a.ix[0, 15]),change1(a.ix[0, 27]),change1(a.ix[0, 39]),change1(a.ix[0, 51]),change1(a.ix[0, 63]),change1(a.ix[0, 75]),change1(a.ix[0, 87]),change1(a.ix[0, 99]),change1(a.ix[0, 111])))
    a5=change2(geomean(change1(a.ix[0, 4]),change1(a.ix[0, 16]),change1(a.ix[0, 28]),change1(a.ix[0, 40]),change1(a.ix[0, 52]),change1(a.ix[0, 64]),change1(a.ix[0, 76]),change1(a.ix[0, 88]),change1(a.ix[0, 100]),change1(a.ix[0, 112])))
    a6=change2(geomean(change1(a.ix[0, 5]),change1(a.ix[0, 17]),change1(a.ix[0, 29]),change1(a.ix[0, 41]),change1(a.ix[0, 53]),change1(a.ix[0, 65]),change1(a.ix[0, 77]),change1(a.ix[0, 89]),change1(a.ix[0, 101])))
    a7=change2(geomean(change1(a.ix[0, 6]),change1(a.ix[0, 18]),change1(a.ix[0, 30]),change1(a.ix[0, 42]),change1(a.ix[0, 54]),change1(a.ix[0, 66]),change1(a.ix[0, 78]),change1(a.ix[0, 90]),change1(a.ix[0, 102])))
    a8=change2(geomean(change1(a.ix[0, 7]),change1(a.ix[0, 19]),change1(a.ix[0, 31]),change1(a.ix[0, 43]),change1(a.ix[0, 55]),change1(a.ix[0, 67]),change1(a.ix[0, 79]),change1(a.ix[0, 91]),change1(a.ix[0, 103])))
    a9=change2(geomean(change1(a.ix[0, 8]),change1(a.ix[0, 20]),change1(a.ix[0, 32]),change1(a.ix[0, 44]),change1(a.ix[0, 56]),change1(a.ix[0, 68]),change1(a.ix[0, 80]),change1(a.ix[0, 92]),change1(a.ix[0, 104])))
    a10=change2(geomean(change1(a.ix[0, 9]),change1(a.ix[0, 21]),change1(a.ix[0, 33]),change1(a.ix[0, 45]),change1(a.ix[0, 57]),change1(a.ix[0, 69]),change1(a.ix[0, 81]),change1(a.ix[0, 93]),change1(a.ix[0, 105])))
    a11=change2(geomean(change1(a.ix[0, 10]),change1(a.ix[0, 22]),change1(a.ix[0, 34]),change1(a.ix[0, 46]),change1(a.ix[0, 58]),change1(a.ix[0, 70]),change1(a.ix[0, 82]),change1(a.ix[0, 94]),change1(a.ix[0, 106])))
    a12=change2(geomean(change1(a.ix[0, 11]),change1(a.ix[0, 23]),change1(a.ix[0, 35]),change1(a.ix[0, 47]),change1(a.ix[0, 59]),change1(a.ix[0, 71]),change1(a.ix[0, 83]),change1(a.ix[0, 95]),change1(a.ix[0, 107])))
    a0 = np.round((a1+a2+a3+a4+a5+a6+a7+a8+a9+a10+a11+a12)/12, 2)
    return a0

In [ ]:
result_mean(per_results(10))

In [ ]:
yield_mean = []
winn = []
losen = []
num = []

In [ ]:
for n in range(3, 50):
    print(n)
    a=per_results(n)
    yield_mean.append(result_mean(a))
    winn.append(win(a))
    num.append('n=%s'%n)
    losen.append(113- win(a))

In [ ]:
b= pd.DataFrame({'n':num, 'mean':yield_mean,'win':winn,'lose':losen}, columns=['n','mean','win','lose'])
c= b.set_index('n')
c

In [ ]:
c.to_excel('per_test.xlsx')